In [ ]:
SELECT SalesOrderID, OrderDate, TotalDue, 
       SUM(TotalDue) OVER(ORDER BY OrderDate
                          ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS RunningTotal
FROM Sales.SalesOrderHeader
ORDER BY OrderDate;


### 1\. Calculate Running Total of Sales

**Objective**: To demonstrate how to calculate the cumulative total of sales over time

This query uses a window function to sum the `TotalDue` from all previous rows up to the current row, partitioned by the order date. It's useful for tracking the progression of sales and can be used in financial reporting or monitoring company performance over time.

In [ ]:
SELECT ProductID, Name, ProductNumber, ListPrice,
       RANK() OVER(PARTITION BY ProductSubcategoryID ORDER BY ListPrice DESC) AS PriceRank
FROM Production.Product
WHERE ProductSubcategoryID IS NOT NULL;


2\. Rank Products by List Price

**Objective**: To rank products within each category based on their list prices in descending order

Here, the `RANK()` function is used to assign a rank to each product within its category based on the list price. This can help in market analysis or inventory management to understand competitive pricing and product positioning.

In [ ]:
SELECT SalesOrderID, OrderDate, TotalDue,
       LAG(TotalDue) OVER(ORDER BY OrderDate, SalesOrderID) AS PreviousSale,
       LEAD(TotalDue) OVER(ORDER BY OrderDate, SalesOrderID) AS NextSale
FROM Sales.SalesOrderHeader;


### 3\. Find Previous and Next Sale for Each Sales Order

**Objective**: To identify preceding and succeeding sales for each order, aiding in sequential data analysis.   
 This query uses `LAG` and `LEAD` to look at the sales amount of the previous and next order. This is valuable for trend analysis or finding sales patterns over time.

In [ ]:
SELECT SalesOrderID, SalesPersonID, OrderDate,
       FIRST_VALUE(SalesOrderID) OVER(PARTITION BY SalesPersonID
                                      ORDER BY OrderDate
                                      ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS FirstOrderID,
       LAST_VALUE(SalesOrderID) OVER(PARTITION BY SalesPersonID
                                     ORDER BY OrderDate
                                     ROWS BETWEEN CURRENT ROW AND UNBOUNDED FOLLOWING) AS LastOrderID
FROM Sales.SalesOrderHeader
WHERE SalesPersonID IS NOT NULL;


### 4\. Get First and Last Sales Order of Each Employee

**Objective**: To determine the first and last sales transactions handled by each salesperson. 

Using `FIRST_VALUE` and `LAST_VALUE`, this query identifies the very first and the most recent sales orders each salesperson processed. Useful for performance review and operational insights.

In [ ]:
SELECT ProductSubcategoryID, SUM(LineTotal) OVER(PARTITION BY ProductSubcategoryID) AS CategorySalesTotal
FROM Sales.SalesOrderDetail AS sod
JOIN Production.Product AS p ON sod.ProductID = p.ProductID
JOIN Production.ProductCategory AS pc ON p.ProductSubcategoryID = pc.ProductCategoryID;


### 5\. Total Sales by Product Category

**Objective**: To calculate the total sales amount for each product category.

This aggregate window function sums up sales by product category, allowing management to see which categories are most lucrative.

In [ ]:
SELECT ProductID,
       [2012] AS Sales2012,
       [2013] AS Sales2013,
       [2014] AS Sales2014
FROM 
(
    SELECT ProductID, YEAR(OrderDate) AS SaleYear, LineTotal
    FROM Sales.SalesOrderDetail sod
    JOIN Sales.SalesOrderHeader soh ON sod.SalesOrderID = soh.SalesOrderID
) AS SourceTable
PIVOT(
    SUM(LineTotal)
    FOR SaleYear IN ([2012], [2013], [2014])
) AS PivotTable;


6\. Pivoting Sales Data

**Objective**: To convert rows into columns, showing annual sales per product for three consecutive years.   
This pivoting example restructures the dataset to show sales totals by year across the columns, facilitating easier year-over-year comparison.

In [ ]:
SELECT SalesPersonID, YEAR(OrderDate) AS SaleYear, SUM(TotalDue) AS TotalSales
FROM Sales.SalesOrderHeader
WHERE SalesPersonID IS NOT NULL
GROUP BY ROLLUP(SalesPersonID, YEAR(OrderDate));


7\. Using ROLLUP to Aggregate Sales Data

**Objective**: To create a hierarchical aggregation of sales data first by salesperson, then by year.

`ROLLUP` generates subtotals at multiple levels, aggregates across different dimensions (here, salesperson and year), and a grand total, enhancing the depth of financial analysis.

In [ ]:
SELECT SalesPersonID,
       SUM(TotalDue) AS TotalSales,
       ROW_NUMBER() OVER(ORDER BY SUM(TotalDue) DESC) AS SalesRank
FROM Sales.SalesOrderHeader
WHERE SalesPersonID IS NOT NULL
GROUP BY SalesPersonID;


### 8\. Ranking Sales Representatives by Sales

**Objective**: To determine the performance ranking of sales representatives based on their total sales.

This uses `ROW_NUMBER` to rank salespeople by their sales achievements, critical for performance evaluations and incentivization strategies.

In [ ]:
SELECT SalesOrderID, TotalDue,
       100.0 * TotalDue / SUM(TotalDue) OVER() AS PercentOfTotalSales
FROM Sales.SalesOrderHeader;


### 9\. Calculate Percentage of Total Sales per Order

**Objective**: To find out what percentage of the total sales is contributed by each individual sale.

 This query calculates each order's contribution to total sales, useful for identifying key transactions or customer importance.

In [ ]:
SELECT SalesOrderID, Attribute, Value
FROM 
(
    SELECT SalesOrderID, CreditCardApprovalCode, Comment
    FROM Sales.SalesOrderHeader
) AS p
UNPIVOT
(
    Value FOR Attribute IN (CreditCardApprovalCode, Comment)
) AS up;


### 10\. Unpivot Sales Data

**Objective**: To transform columns into rows, reversing the typical pivot table format.   

The `UNPIVOT` function takes multiple column-based data and converts it into rows, making the data suitable for certain types of analysis or reporting.